In [1]:
import importlib
import models
import helper_functions
import preprocess
import settings

importlib.reload(settings)
# importlib.reload(models)
importlib.reload(helper_functions)
importlib.reload(preprocess)

from models.LSTM import LSTM_HumanFi, CNN_LSTM, CNN_BiLSTM_TemporalAttention, CNN_BiLSTM_ChannelAttention, CNN_BiLSTM_DualAttention, CNN_BiLSTM_Attention
from models.RadioNet import RadioNet_NeuralWave
from models.ResNet import ECAResNet1D, ECABasicBlock1D, ResNet1D_JARILWWF, OptResNet1D_JARILWWF, OptECAResNet1D_JARILWWF, CustomResNet1D, CustomECAResNet1D
from models.TemporalConvNet import TemporalConvNet

from helper_functions import grid_search, run_gridsearch_with_seeds
from preprocess import DataPreprocessor
from settings import folder_path_5ghz_10hz_collected, folder_path_60ghz_collected, folder_path_5ghz_200hz_collected, folder_path_60ghz_external, output_path

In [2]:
input_dim = 60
num_epochs = 500

param_grid = {
    'batch_size': [32],
    'learning_rate': [0.0007], # 0.001,
    'optimizer': ['adam'],
    'mixup_alpha': [0.0, 0.4],
    'smoothing_prob': [0.0, 0.5],
    'model': [],
}

hidden_dims = [64, 128] # 32, 256
num_layers_list = [1, 2]
bidirectional_flags = [False, True]
dropouts = [0.2, 0.5]

for hidden_dim in hidden_dims:
    for num_layers in num_layers_list:
        for bidirectional in bidirectional_flags:
            for dropout in dropouts:
                param_grid['model'].append({
                    'model_class': LSTM_HumanFi,
                    'model_args': {
                        'input_dim': input_dim,
                        'hidden_dim': hidden_dim,
                        'num_layers': num_layers,
                        'bidirectional': bidirectional,
                        'dropout': dropout
                    },
                    'num_epochs': num_epochs
                })

tcn_channel_options = [
    # [32, 64],
    [64, 128],
    [64, 128, 128],
    [64, 128, 256],
    # [128, 256, 512],
]
tcn_kernel_sizes = [2, 3] # 5
tcn_dropout_rates = [0.2, 0.5]

for channels in tcn_channel_options:
    for kernel_size in tcn_kernel_sizes:
        for dropout in tcn_dropout_rates:
            param_grid['model'].append({
                'model_class': TemporalConvNet,
                'model_args': {
                    'num_inputs': input_dim,
                    'num_channels': channels,
                    'kernel_size': kernel_size,
                    'dropout': dropout
                },
                'num_epochs': num_epochs
            })

cnn_lstm_hidden_dims = [64, 128] # 256
cnn_lstm_layers = [1, 2]

for hidden_dim in cnn_lstm_hidden_dims:
    for num_layers in cnn_lstm_layers:
        param_grid['model'].append({
            'model_class': CNN_BiLSTM_Attention,
            'model_args': {
                'input_dim': input_dim,
                'cnn_filters': 64,
                'lstm_units': hidden_dim,
                'num_layers': num_layers,
            },
            'num_epochs': num_epochs
        })

        param_grid['model'].append({
            'model_class': CNN_BiLSTM_TemporalAttention,
            'model_args': {
                'input_dim': input_dim,
                'cnn_channels': 64,
                'lstm_hidden_dim': hidden_dim,
                'lstm_layers': num_layers,
            },
            'num_epochs': num_epochs
        })

        # param_grid['model'].append({
        #     'model_class': CNN_BiLSTM_ChannelAttention,
        #     'model_args': {
        #         'input_dim': input_dim,
        #         'cnn_channels': 64,
        #         'lstm_hidden_dim': hidden_dim,
        #         'lstm_layers': num_layers,
        #     },
        #     'num_epochs': num_epochs
        # })
        #
        # param_grid['model'].append({
        #     'model_class': CNN_BiLSTM_DualAttention,
        #     'model_args': {
        #         'input_dim': input_dim,
        #         'cnn_channels': 64,
        #         'lstm_hidden_dim': hidden_dim,
        #         'lstm_layers': num_layers,
        #     },
        #     'num_epochs': num_epochs
        # })


param_grid['model'] += [
    # {
    #     'model_class': RadioNet_NeuralWave,
    #     'model_args': {'input_dim': input_dim},
    #     'num_epochs': num_epochs,
    #     'data_preprocessor': DataPreprocessor(target_dim=354)
    # },
    
    # the ones commented below are not optimal (proven by early tests)
    # {
    #     'model_class': ECAResNet1D,
    #     'model_args': {'input_channels': input_dim, 'block': ECABasicBlock1D, 'layers': (1, 1, 1, 1)},
    #     'num_epochs': num_epochs
    # },
    # {
    #     'model_class': ECAResNet1D,
    #     'model_args': {'input_channels': input_dim, 'block': ECABasicBlock1D, 'layers': (2, 2, 2, 2)},
    #     'num_epochs': num_epochs
    # },
    # {
    #     'model_class': ResNet1D_JARILWWF,
    #     'model_args': {'input_channels': input_dim, 'layers': [1,1,1,1]},
    #     'num_epochs': num_epochs,
    # },
    # {
    #     'model_class': ResNet1D_JARILWWF,
    #     'model_args': {'input_channels': input_dim, 'layers': [2,2,2,2]},
    #     'num_epochs': num_epochs,
    # },
    
    
    {
        'model_class': OptResNet1D_JARILWWF,
        'model_args': {'input_channels': input_dim, 'layers': [1,1,1,1]},
        'num_epochs': num_epochs,
    },
    # {
    #     'model_class': OptResNet1D_JARILWWF,
    #     'model_args': {'input_channels': input_dim, 'layers': [2,2,2,2]},
    #     'num_epochs': num_epochs,
    # },
    {
        'model_class': OptECAResNet1D_JARILWWF,
        'model_args': {'input_channels': input_dim, 'layers': [1,1,1,1]},
        'num_epochs': num_epochs,
    },
    # {
    #     'model_class': OptECAResNet1D_JARILWWF,
    #     'model_args': {'input_channels': input_dim, 'layers': [2,2,2,2]},
    #     'num_epochs': num_epochs,
    # },
    {
        'model_class': CustomResNet1D,
        'model_args': {'input_channels': input_dim},
        'num_epochs': num_epochs,
    },
    {
        'model_class': CustomECAResNet1D,
        'model_args': {'input_channels': input_dim},
        'num_epochs': num_epochs,
    },
] 

In [3]:
gridsearch_results_df = run_gridsearch_with_seeds(
    gridsearch_func=grid_search,
    gridsearch_args={
        'param_grid': param_grid,
        'folder_path': folder_path_60ghz_collected,
        'background_subtraction': False,
        'seconds_per_sample': 5,
        'rows_per_second': 10,
    },
    n_seeds=3,
    output_dir=output_path,
    filename="gridsearch_60ghz_coll"
)


--- Running seed 42 ---
Running 160 configurations...

Processing Model: LSTM_HumanFi with params {'input_dim': 60, 'hidden_dim': 64, 'num_layers': 1, 'bidirectional': False, 'dropout': 0.2}
Started loading data...
Train size: 384, Validation size: 68, Test size: 68
Train labels distribution: Counter({10: 21, 0: 20, 2: 20, 3: 20, 15: 20, 16: 20, 17: 20, 18: 20, 1: 19, 4: 19, 5: 19, 7: 19, 8: 19, 9: 19, 12: 19, 13: 19, 14: 19, 6: 18, 11: 17, 19: 17})
Validation labels distribution: Counter({0: 4, 1: 4, 4: 4, 8: 4, 10: 4, 12: 4, 13: 4, 14: 4, 2: 3, 3: 3, 5: 3, 6: 3, 7: 3, 9: 3, 11: 3, 15: 3, 16: 3, 17: 3, 18: 3, 19: 3})
Test labels distribution: Counter({0: 4, 1: 4, 4: 4, 8: 4, 10: 4, 12: 4, 13: 4, 14: 4, 2: 3, 3: 3, 5: 3, 6: 3, 7: 3, 9: 3, 11: 3, 15: 3, 16: 3, 17: 3, 18: 3, 19: 3})
Done loading data (batch size: 32) in 0:00:00.091676.

Testing Configuration 0: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.0

(Device: cuda) Started training model: LSTM_Huma

C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Early stopping at epoch 170 (no improvement in 50 epochs)
Training time:  0:02:28.992482


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 65: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 218 (no improvement in 50 epochs)
Training time:  0:03:07.656256


...


Testing Configuration 109: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 188 (no improvement in 50 epochs)
Training time:  0:07:23.040152


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 110: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.4, smoothing_prob=0.0

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 383 (no improvement in 50 epochs)
Training time:  0:14:59.623346


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 111: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.4, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 312 (no improvement in 50 epochs)
Training time:  0:12:16.738456

Processing Model: CNN_BiLSTM_Attention with params {'input_dim': 60, 'cnn_filters': 64, 'lstm_units': 64, 'num_layers': 1}
Started loading data...
Train size: 384, Validation size: 68, Test size: 68
Train labels distribution: Counter({10: 21, 0: 20, 2: 20, 3: 20, 15: 20, 16: 20, 17: 20, 18: 20, 1: 19, 4: 19, 5: 19, 7: 19, 8: 19, 9: 19, 12: 19, 13: 19, 14: 19, 6: 18, 11: 17, 19: 17})
Validation labels distribution: Counter({0: 4, 1: 4, 4: 4, 8: 4, 10: 4, 12: 4, 13: 4, 14: 4, 2: 3, 3: 3, 5: 3, 6: 3, 7: 3, 9: 3, 11: 3, 15: 3, 16: 3, 17: 3, 18: 3, 19: 3})
Test labels distribution: Counter({0: 4, 1: 4, 4: 4, 8: 4, 10: 4, 12: 4, 13: 4, 14: 4, 2: 3, 3: 3, 5: 3, 6: 3, 7: 3, 9: 3, 11: 3, 15: 3, 16: 3, 17: 3, 18: 3, 19: 3})
Done loading data (batch

In [3]:
gridsearch_results_df_bgsub = run_gridsearch_with_seeds(
    gridsearch_func=grid_search,
    gridsearch_args={
        'param_grid': param_grid,
        'folder_path': folder_path_60ghz_collected,
        'background_subtraction': True,
        'seconds_per_sample': 5,
        'rows_per_second': 10,
    },
    n_seeds=3,
    output_dir=output_path,
    filename="gridsearch_60ghz_coll"
)


--- Running seed 42 ---
Running 160 configurations...

Processing Model: LSTM_HumanFi with params {'input_dim': 60, 'hidden_dim': 64, 'num_layers': 1, 'bidirectional': False, 'dropout': 0.2}
Started loading data...
Train size: 384, Validation size: 68, Test size: 68
Train labels distribution: Counter({10: 21, 0: 20, 2: 20, 3: 20, 15: 20, 16: 20, 17: 20, 18: 20, 1: 19, 4: 19, 5: 19, 7: 19, 8: 19, 9: 19, 12: 19, 13: 19, 14: 19, 6: 18, 11: 17, 19: 17})
Validation labels distribution: Counter({0: 4, 1: 4, 4: 4, 8: 4, 10: 4, 12: 4, 13: 4, 14: 4, 2: 3, 3: 3, 5: 3, 6: 3, 7: 3, 9: 3, 11: 3, 15: 3, 16: 3, 17: 3, 18: 3, 19: 3})
Test labels distribution: Counter({0: 4, 1: 4, 4: 4, 8: 4, 10: 4, 12: 4, 13: 4, 14: 4, 2: 3, 3: 3, 5: 3, 6: 3, 7: 3, 9: 3, 11: 3, 15: 3, 16: 3, 17: 3, 18: 3, 19: 3})
Done loading data (batch size: 32) in 0:00:00.093973.

Testing Configuration 0: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.0

(Device: cuda) Started training model: LSTM_Huma

C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Early stopping at epoch 110 (no improvement in 50 epochs)
Training time:  0:01:32.405313


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 65: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 104 (no improvement in 50 epochs)
Training time:  0:01:26.479996


...


Testing Configuration 109: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.0, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 144 (no improvement in 50 epochs)
Training time:  0:05:40.110850


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 110: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.4, smoothing_prob=0.0

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 212 (no improvement in 50 epochs)
Training time:  0:08:17.521914


C:\Users\centu\PycharmProjects\MasterThesis_mmWavePI\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)



Testing Configuration 111: batch_size=32, lr=0.0007, optimizer=adam, mixup_alpha=0.4, smoothing_prob=0.5

(Device: cuda) Started training model: TemporalConvNet
Early stopping at epoch 235 (no improvement in 50 epochs)
Training time:  0:09:11.694839

Processing Model: CNN_BiLSTM_Attention with params {'input_dim': 60, 'cnn_filters': 64, 'lstm_units': 64, 'num_layers': 1}
Started loading data...
Train size: 384, Validation size: 68, Test size: 68
Train labels distribution: Counter({10: 21, 0: 20, 2: 20, 3: 20, 15: 20, 16: 20, 17: 20, 18: 20, 1: 19, 4: 19, 5: 19, 7: 19, 8: 19, 9: 19, 12: 19, 13: 19, 14: 19, 6: 18, 11: 17, 19: 17})
Validation labels distribution: Counter({0: 4, 1: 4, 4: 4, 8: 4, 10: 4, 12: 4, 13: 4, 14: 4, 2: 3, 3: 3, 5: 3, 6: 3, 7: 3, 9: 3, 11: 3, 15: 3, 16: 3, 17: 3, 18: 3, 19: 3})
Test labels distribution: Counter({0: 4, 1: 4, 4: 4, 8: 4, 10: 4, 12: 4, 13: 4, 14: 4, 2: 3, 3: 3, 5: 3, 6: 3, 7: 3, 9: 3, 11: 3, 15: 3, 16: 3, 17: 3, 18: 3, 19: 3})
Done loading data (batch